# Взаимодействие между процессами

Рассмотрим, как сделать так, чтобы процессы узнали друг о друге и смогли взаимодействовать между собой.
Заодно рассмотрим применение SimPy в парадигме объектно-ориентированного программирования (с классами).

## Программная реализация

Рассмотрим вопрос моделирования электромобиля, способного лишь ехать в течение 2 единиц времени и заряжаться в течение 5 единиц времени.
Но также есть водитель, который может прервать зарядку автомобиля и начать движение.

Компьютерная реализация представлена ниже:

In [7]:
import simpy as sim


# На этот раз автомобиль представим как объект (класс)
class Car:
    # При инициализации передадим экземпляр среды, ...
    def __init__(self, env: sim.Environment):
        self.env = env
        # а также инициализируем основной процесс этого класса
        self.action = env.process(self.run())

    # Функция-генератор процесса класса
    def run(self):
        while True:
            print(f'{self.env.now}: Паркуется и заряжается')
            charge_duration = 5
            # Блок try-except здесь позволяет "мягко" прервать
            # ожидание события "зарядка завершена"
            try:
                # Обратите внимание: внутри процесса run инициализирован
                # новый процесс charge, причём оператор yield говорит,
                # что мы будем ждать завершения этого процесса
                yield self.env.process(self.charge(charge_duration))
            except sim.Interrupt:
                # Если ожидание прервано, то выводится это сообщение
                print(f'{self.env.now}: Зарядка прервана!')

            print(f'{self.env.now}: Едет')
            trip_duration = 2
            # Этот yield в блок try-except не обрамлён,
            # поэтому если будет прервано ожидание этого события,
            # то рухнет вся программа,
            # т.к. никто не поймает исключение sim.Interrupt
            yield self.env.timeout(trip_duration)

    # Процесс зарядки
    def charge(self, duration):
        # Просто ожидание в течение заданного времени
        yield self.env.timeout(duration)


# Отдельный от класса Car процесс,
# моделирующий поведение водителя
def driver(env, car):
    # Водитель единожды ждёт 3 единицы времени, ...
    yield env.timeout(3)
    # после чего прерывает процесс (зарядки) автомобиля
    # путём проброса исключения типа sim.Interrupt
    car.action.interrupt()
    # !!! ВАЖНО !!!
    # Это учебный пример и мы точно знаем, что через 3 ед. времени
    # после начала симуляции автомобиль заряжается.
    # Если бы он ехал, то мы получили бы упавшую программу,
    # т.к. в этом случае исключение sim.Interrupt не было бы
    # перехвачено в блоке try-except


# Создаём экземпляр среды
env = sim.Environment()
# Создаём экземпляр электромобиля,
# при этом автоматически инициализируется его процесс run
car = Car(env)
# Инициализируем процесс водителя
env.process(driver(env, car))
# Запускаем моделирование
env.run(until=15)

0: Паркуется и заряжается
3: Зарядка прервана!
3: Едет
5: Паркуется и заряжается
10: Едет
12: Паркуется и заряжается


В сущности приведённый код мало отличается от такового из раздела {doc}`basics` (метод `run` почти совпадает с функцией `car`, код в глобальной области видимости также почти не изменился).
Рассмотри новинки и особенности.

**Во-первых**, класс `Car` имеет два процесса: `run` и `charge`, причём последний инициализируется внутри первого.
Это — одна из разновидностей взаимодействия между процессами: процесс может порождать процесс, а также ждать его завершения.
Как раз в `run` мы не только инициализируем процесс `charge`, но и ждём его завершения, о чём говорит наличие оператора `yield`.
Сам процесс `run` запускается при инициализации экземпляра `car = Car(env)`.

**Во-вторых**, рассматриваемый участок кода обрамлён в блок `try-except`, что позволяет перехватывать исключения по типу `sim.Interrupt`, которое возникает, когда один процесс прерывает другой.
Прерывание — ещё один способ взаимодействия процессов.

```{important}
Второй оператор `yield` в методе `run` не обрамлён обработчиком исключений.
Поэтому процесс езды автомобиля не может быть прерван.
Если вдруг окажется, что `self.action` в процессе поездки, а не зарядки, то при прерывании `car.action.interrupt()` исключение `sim.Interrupt` перехвачено не будет.
Как результат, рухнет вся программа.
```

**В-третьих**, есть функция `driver`, моделирующая водителя, прерывающего процесс зарядки автомобиля.
В этой функции нет какого-либо цикла, поэтому после события `yield env.timeout(3)`, процесс `driver` завершается, возвращая `return None` (явно не указано, но Python делает это по умолчанию).
Данный процесс запускается в глобальной области `env.process(driver(env, car))`.

## Заключение

Взаимодействие между процессами в SimPy реализуется довольно просто.
Возможно порождение процесса и событий внутри других процессов, ожидание окончания процесса другим процессом и прерывание процессов.
Единственное, осторожного программирования требует реализация прерывания процессов, чтобы не допустить аварийного завершения программы.